# First eval

This jupyter notebook is for trying out what research I have read and my ideas in this [Notion](https://www.notion.so/Model-evaluation-170d3cfe027d80889de3cfbb35b531a4).

I played around with a Mistral Large model to build a mathematics chatbot. It is simple and I prompt engineered it. I might change the model to a smaller and cheaper one, but I still want to use a model that is considerd good enough. My plan is to use three small open source models to evaluate the mahtematics model. I'll also use different models for the mathematics chatbot and for the evaluation models to get realible results.

I'll use this [dataset](https://github.com/google-deepmind/mathematics_dataset?tab=readme-ov-file) as a test data set. I'll select 50 mathematics questiosn from the dataset. Then I'll ask the mathematicsbot the same questions. 
Then I'll let: 
- First a Mistral Large model compare the answers. Same model as in the mathematicsbot but different prompt engineering. 
    - This is not a good way to evaluate models, but I wish to try it. Espeacilly when it is only 50 questions it is manageble for me to compare the answers.
- Then I'll let three smaller open-source models compare the answers.
- After that I'll add a fourth model to compare the answers of the three open-open source model to give a final evaluation about the mathematics chat bot.

In [61]:
!pip3 install mistralai
!pip3 install python-dotenv
!pip3 install mathematics_dataset
!pip3 install pip install sympy==1.6
!pip3 install numpy==1.24.3
!pip3 install json


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


I've used the following prompt for the Mistral Large 2.1 model
"Your task is to help solve math problems. Generate three answers that helps solving the problem but doesn't give the solution. When generating the 4th answer to the problem you can give the solution. When you don't know the answer say that you don't know the answer. Write the mathematical part of the answer in latex."

In [2]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

client = Mistral(api_key=api_key)

chat_response = client.agents.complete(
    agent_id= agent_id,
    messages=[
        {
            "role": "user",
            "content": """Given the matrix:
                            A = [
                                [4, 1, -2],
                                [1, 3,  0],
                                [-2, 0, 2]
                            ]

                            Tasks:
                            1. Compute the eigenvalues of A.
                            2. Compute the eigenvectors of A for each eigenvalue.
                            3. Verify that for each eigenpair (λ, v), the equation A * v = λ * v holds.
                            4. If possible, diagonalize A by finding matrices P (eigenvectors) and D (diagonal matrix of eigenvalues) such that A = P * D * P⁻¹.
                            """
        },
    ],
)
print(chat_response.choices[0].message.content)

1. To compute the eigenvalues of the matrix $A$, you need to solve the characteristic equation $\det(A - \lambda I) = 0$, where $I$ is the identity matrix. This involves finding the roots of the characteristic polynomial.

2. For each eigenvalue $\lambda$ found in step 1, solve the equation $(A - \lambda I)v = 0$ to find the corresponding eigenvectors $v$. This involves solving a homogeneous system of linear equations for each eigenvalue.

3. To verify that for each eigenpair $(\lambda, v)$, the equation $A * v = \lambda * v$ holds, substitute the eigenvalue $\lambda$ and the corresponding eigenvector $v$ into the equation and check if the left-hand side equals the right-hand side.

4. To diagonalize the matrix $A$, we need to form the matrix $P$ whose columns are the eigenvectors of $A$, and the diagonal matrix $D$ whose diagonal elements are the eigenvalues of $A$. Then, we can check if $A = P * D * P^{-1}$.

   For the given matrix $A = \begin{bmatrix}
   4 & 1 & -2 \\
   1 & 3 & 0 

In [70]:
import json
import re

# Assuming 'output' is fetched from your command
output = !python -m mathematics_dataset.generate

# Initialize variables for parsing
cleaned_output = []

current_line = ""  # To accumulate multi-line questions

# Regular expression pattern to remove color codes (ANSI escape sequences)
color_code_pattern = re.compile(r'\u001b\[[0-9;]*m')

# Iterate over the lines of the output
for line in output:
    line = line.strip()  # Remove leading and trailing whitespace
    
    # Skip subheadings (lines without spaces and that don't end with '.' or '?')
    if len(line) > 0 and " " not in line and not (line.endswith(".") or line.endswith("?")):
        continue

    if line:  # Skip empty lines
        # Check if the line ends with a period (.) or a question mark (?)
        if "." in line or "?" in line:
            # Append the line to the current question
            current_line += " " + line.strip()
        else:
            # Accumulate lines that are part of a multi-line question
            current_line += " " + line.strip()
        
        # If the current line contains a question and answer, process it
        if "  " in current_line:  # Check for the delimiter between question and answer
            question, answer = current_line.rsplit("  ", 1)  # Split by the last occurrence of "  "
            
            # Remove color codes from the answer
            answer = re.sub(color_code_pattern, '', answer).strip()

            cleaned_output.append({
                "question": question.strip(),
                "answer": answer
            })  # Store the question-answer pair
            current_line = ""  # Reset for the next question

# Now, save the cleaned output to a JSON file
with open('questions_and_answers.json', 'w') as json_file:
    json.dump(cleaned_output, json_file, indent=4)

print("Output saved to output.json")

Output saved to output.json


In [34]:
!python -m mathematics_dataset.generate --filter=linear_1d

train/algebra__linear_1d
 Solve 6*w = 245*w + 1912 for w.  -8
 Solve 0 = 225*t + 6015 + 1405 + 3155 for t.  -47
 Solve -2*p - 12*p - 139*p + 2448 = 0 for p.  16
 Solve 222*g - 7760 = -258*g + 4720 for g.  26
 Solve 0 = -5*j + 3 + 7 for j.  2
 Solve -31 = -0*g - 3*g + 34*g for g.  -1
 Solve 380*c - 376*c = 8 for c.  2
 Solve 4 = -3*h + 4*h for h.  4
 Solve -2*q = -4*q + 3*q for q.  0
 Solve 5 = -4*k + 1 for k.  -1
train/algebra__linear_1d_composed
 Suppose 4*c - 2 = 4*j - 6, 0 = -c + 1. Suppose j*u - 3 = -u. Let i be (2 +
  -4)*2*-1. Solve -i = -y + u for y.  5
 Let s be (0 + (56 - 8))*2/4. Let t be (-2)/s*2 - 975/(-234). Solve -l - t = 1
  for l.  -5
 Suppose -65 = 8*p - 13*p. Suppose 3 = -2*l + p. Suppose 0 = l*o - 41 - 4. Solve
  -4*z = -o*z for z.  0
 Let j = -76 + 76. Let b be (-10)/(-30)*(18 + j). Solve 5*u = -b - 4 for u.
  -2
 Suppose 276 = 243*o - 237*o. Suppose 2*i - 2*a = o, -a = -4*i + a + 88. Solve
  i*k - 23*k = 0 for k.  0
 Let j(g) = -g**3 - 11*g**2 + g + 13. Let s be j(